[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/dartboard.ipynb)

# Dartboard RAG：具有平衡相关性和多样性的检索增强生成

## 概述
**Dartboard RAG** 流程解决了大型知识库中的一个常见挑战：确保检索到的信息既相关又非冗余。通过明确优化组合的相关性-多样性评分函数，它防止了多个 top-k 文档提供相同的信息。这种方法源于论文中的优雅方法：

> [*使用相关信息增益实现更好的 RAG*](https://arxiv.org/abs/2407.12101)

该论文概述了核心思想的三种变体——混合 RAG（密集+稀疏）、交叉编码器版本和普通方法。**普通方法**最直接地传达了基本概念，而此实现通过可选权重对其进行了扩展，以控制相关性和多样性之间的平衡。

## 动机

1. **密集、重叠的知识库**  
   在大型数据库中，文档可能会重复相似的内容，导致 top-k 检索中出现冗余。

2. **改进的信息覆盖范围**  
   结合相关性和多样性可以产生更丰富的文档集，从而减轻内容过于相似的“回声室”效应。


## 关键组件

1. **相关性与多样性的结合**  
   - 计算一个综合得分，该得分既考虑了文档与查询的相关性，也考虑了其与已选文档的区别。

2. **加权平衡**  
   - 引入 RELEVANCE_WEIGHT 和 DIVERSITY_WEIGHT 以允许动态控制评分。  
   - 有助于避免过于多样化但相关性较低的结果。

3. **生产就绪代码**  
   - 源自官方实现，但为清晰起见进行了重组。  
   - 允许更轻松地集成到现有的 RAG 管道中。

## 方法细节

1. **文档检索**  
   - 基于相似性（例如，余弦或 BM25）获取一组初始候选文档。  
   - 通常检索 top-N 候选文档作为起点。

2. **评分与选择**  
   - 每个文档的总分结合了**相关性**和**多样性**：  
   - 选择得分最高的文档，然后对与其过于相似的文档进行惩罚。  
   - 重复此过程，直到确定 top-k 文档。

3. **混合/融合与交叉编码器支持**  
   基本上，您只需要文档与查询之间的距离，以及文档之间的距离。您可以轻松地从混合/融合检索或交叉编码器检索中提取这些信息。我唯一的建议是减少对基于排序的得分的依赖。
   - 对于**混合/融合检索**：将相似性（密集和稀疏/BM25）合并为单个距离。这可以通过组合密集和稀疏向量上的余弦相似性（例如，对它们进行平均）来实现。转换为距离很简单（1 - 平均余弦相似性）。 
   - 对于**交叉编码器**：您可以直接使用交叉编码器相似性得分（1 - 相似性），并可能使用缩放因子进行调整。

4. **平衡与调整**  
   - 根据您的需求和数据集的密度调整 DIVERSITY_WEIGHT 和 RELEVANCE_WEIGHT。  



通过将**相关性**和**多样性**都集成到检索中，Dartboard RAG 方法可确保 top-k 文档共同提供更丰富、更全面的信息，从而在检索增强生成系统中获得更高质量的响应。

该论文还有一个官方代码实现，此代码基于该实现，但我认为这里的代码更具可读性、可管理性且已为生产就绪。

# 包安装和导入

下面的单元格安装运行此笔记本所需的所有必要包。


In [ ]:
# 安装所需的包
!pip install numpy python-dotenv

In [ ]:
# 克隆仓库以访问辅助函数和评估模块
!git clone https://github.com/NirDiamant/RAG_TECHNIQUES.git
import sys
sys.path.append('RAG_TECHNIQUES')
# 如果您需要使用最新数据运行
# !cp -r RAG_TECHNIQUES/data .

In [ ]:
import os
import sys
from dotenv import load_dotenv
from scipy.special import logsumexp
from typing import Tuple, List, Any
import numpy as np

# 从 .env 文件加载环境变量
load_dotenv()
# 设置 OpenAI API 密钥环境变量（如果不使用 OpenAI，请注释掉）
if not os.getenv('OPENAI_API_KEY'):
    print("Please enter your OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = input("Please enter your OpenAI API key: ")
else:
    os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# 为兼容 Colab 替换了原始路径追加
from helper_functions import *
from evaluation.evalute_rag import *


Please enter your OpenAI API key: 


### 读取文档

In [ ]:
# 下载所需的数据文件
import os
os.makedirs('data', exist_ok=True)

# 下载本笔记本使用的 PDF 文档
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf


In [3]:
path = "data/Understanding_Climate_Change.pdf"

### 编码文档

In [4]:
# 这部分与 simple_rag.ipynb 相同，只是模拟了一个密集数据集
def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    使用 OpenAI 嵌入将 PDF 书籍编码到向量存储中。

    参数：
        path: PDF 文件的路径。
        chunk_size: 每个文本块的期望大小。
        chunk_overlap: 连续块之间的重叠量。

    返回：
        包含编码后的书籍内容的 FAISS 向量存储。
    """

    # 加载 PDF 文档
    loader = PyPDFLoader(path)
    documents = loader.load()
    documents=documents*5 # 每个文档加载 5 次以模拟密集数据集

    # 将文档拆分为块
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # 创建嵌入（已使用 OpenAI 和 Amazon Bedrock 测试）
    embeddings = get_langchain_embedding_provider(EmbeddingProvider.OPENAI)
    #embeddings = get_langchain_embedding_provider(EmbeddingProvider.AMAZON_BEDROCK) #亚马逊基岩嵌入

    # 创建向量存储
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore

### 创建向量存储


In [5]:
chunks_vector_store = encode_pdf(path, chunk_size=1000, chunk_overlap=200)

### 一些用于使用向量存储进行检索的辅助函数。
这部分与 simple_rag.ipynb 相同，只是它使用的是实际的 FAISS 索引（而不是包装器）

In [6]:

def idx_to_text(idx:int):
    """
    将向量存储索引转换为相应的文本。
    """
    docstore_id = chunks_vector_store.index_to_docstore_id[idx]
    document = chunks_vector_store.docstore.search(docstore_id)
    return document.page_content


def get_context(query:str,k:int=5) -> Tuple[np.ndarray, np.ndarray, List[str]]:
    """
    使用 top k 检索来检索查询的前 k 个上下文项。
    """
    # 常规 top k 检索
    q_vec=chunks_vector_store.embedding_function.embed_documents([query])
    _,indices=chunks_vector_store.index.search(np.array(q_vec),k=k)

    texts = [idx_to_text(i) for i in indices[0]]
    return texts


In [10]:

test_query = "What is the main cause of climate change?"


### 常规 top k 检索
- 此演示表明，当数据库密集时（此处我们通过加载每个文档 5 次来模拟密度），结果不佳，我们无法获得最相关的结果。请注意，前 3 个结果都是同一文档的重复。

In [11]:
texts=get_context(test_query,k=3)
show_context(texts)

Context 1:
driven by human activities, particularly the emission of greenhou se gases.  
Chapter 2: Causes of Climate Change  
Greenhouse Gases  
The primary cause of recent climate change is the increase in greenhouse gases in the 
atmosphere. Greenhouse gases, such as carbon dioxide (CO2), methane (CH4), and nitrous 
oxide (N2O), trap heat from the sun, creating a "greenhouse effect." This effect is  essential 
for life on Earth, as it keeps the planet warm enough to support life. However, human 
activities have intensified this natural process, leading to a warmer climate.  
Fossil Fuels  
Burning fossil fuels for energy releases large amounts of CO2. This includes coal, oil, and 
natural gas used for electricity, heating, and transportation. The industrial revolution marked 
the beginning of a significant increase in fossil fuel consumption, which continues to rise 
today.  
Coal


Context 2:
driven by human activities, particularly the emission of greenhou se gases.  
Chapter 2: C

## 现在是真正的部分 :)


### 更多用于距离归一化的工具

In [21]:
def lognorm(dist:np.ndarray, sigma:float):
    """
    计算给定距离和 sigma 的对数正态概率。
    """
    if sigma < 1e-9: 
        return -np.inf * dist
    return -np.log(sigma) - 0.5 * np.log(2 * np.pi) - dist**2 / (2 * sigma**2)


## 贪心 Dartboard 搜索

这是核心算法：一种搜索算法，通过平衡两个因素从集合中选择一组多样化的相关文档：与查询的相关性和所选文档之间的多样性。

给定查询与文档之间的距离，以及所有文档之间的距离，该算法：

1. 首先选择最相关的文档
2. 通过组合以下内容迭代选择其他文档：
   - 与原始查询的相关性
   - 与先前选择的文档的多样性

相关性和多样性之间的平衡由权重控制：
- `DIVERSITY_WEIGHT`：与现有选择差异的重要性
- `RELEVANCE_WEIGHT`：与查询相关性的重要性
- `SIGMA`：用于概率转换的平滑参数

该算法返回所选文档及其选择分数，这使其对于需要相关但多样化结果的搜索结果等应用程序非常有用。

例如，在搜索新闻文章时，它会首先返回最相关的文章，然后查找既切题又提供新信息的文章，从而避免冗余选择。

In [ ]:
# 配置参数
DIVERSITY_WEIGHT = 1.0  # 文档选择中多样性的权重
RELEVANCE_WEIGHT = 1.0  # 与查询相关性的权重
SIGMA = 0.1  # 概率分布的平滑参数

def greedy_dartsearch(
    query_distances: np.ndarray,
    document_distances: np.ndarray,
    documents: List[str],
    num_results: int
) -> Tuple[List[str], List[float]]:
    """
    执行贪心 dartboard 搜索以选择 top k 个平衡相关性和多样性的文档。
    
    参数：
        query_distances: 查询与每个文档之间的距离
        document_distances: 文档之间的成对距离
        documents: 文档文本列表
        num_results: 要返回的文档数
    
    返回：
        包含以下内容的元组：
        - 所选文档文本列表
        - 每个文档的选择分数列表
    """
    # 避免在概率计算中除以零
    sigma = max(SIGMA, 1e-5)
    
    # 将距离转换为概率分布
    query_probabilities = lognorm(query_distances, sigma)
    document_probabilities = lognorm(document_distances, sigma)
    
    # 使用最相关的文档进行初始化
    
    most_relevant_idx = np.argmax(query_probabilities)
    selected_indices = np.array([most_relevant_idx])
    selection_scores = [1.0] # 第一个文档的虚拟分数
    # 从第一个选定的文档中获取初始距离
    max_distances = document_probabilities[most_relevant_idx]
    
    # 选择剩余的文档
    while len(selected_indices) < num_results:
        # 考虑新文档更新最大距离
        updated_distances = np.maximum(max_distances, document_probabilities)
        
        # 计算组合的多样性和相关性得分
        combined_scores = (
            updated_distances * DIVERSITY_WEIGHT +
            query_probabilities * RELEVANCE_WEIGHT
        )
        
        # 归一化分数并屏蔽已选择的文档
        normalized_scores = logsumexp(combined_scores, axis=1)
        normalized_scores[selected_indices] = -np.inf
        
        # 选择最佳剩余文档
        best_idx = np.argmax(normalized_scores)
        best_score = np.max(normalized_scores)
        
        # 更新跟踪变量
        max_distances = updated_distances[best_idx]
        selected_indices = np.append(selected_indices, best_idx)
        selection_scores.append(best_score)
    
    # 返回选定的文档及其分数
    selected_documents = [documents[i] for i in selected_indices]
    return selected_documents, selection_scores

## Dartboard 上下文检索

### 使用 dartboard 检索的主要功能。它取代了 get_context（即简单的 RAG）。它：

1. 获取文本查询，将其向量化，通过简单的 RAG 获取 top k 个文档（及其向量）
2. 使用这些向量计算与查询的相似度以及候选匹配项之间的相似度
3. 运行 dartboard 算法将候选匹配项优化为 k 个文档的最终列表
4. 返回最终的文档列表及其分数

In [ ]:

def get_context_with_dartboard(
    query: str,
    num_results: int = 5,
    oversampling_factor: int = 3
) -> Tuple[List[str], List[float]]:
    """
    使用 dartboard 算法检索查询最相关和最多样化的上下文项。
    
    参数：
        query: 搜索查询字符串
        num_results: 要返回的上下文项数（默认值：5）
        oversampling_factor: 用于对初始结果进行过采样以获得更好多样性的因子（默认值：3）
    
    返回：
        包含以下内容的元组：
        - 选定的上下文文本列表
        - 选择分数列表
        
    注意：
        该函数使用转换为距离的余弦相似度。初始检索 
        获取 oversampling_factor * num_results 项以确保最终选择中足够的多样性。
    """
    # 嵌入查询并检索初始候选
    query_embedding = chunks_vector_store.embedding_function.embed_documents([query])
    _, candidate_indices = chunks_vector_store.index.search(
        np.array(query_embedding),
        k=num_results * oversampling_factor
    )
    
    # 获取候选的文档向量和文本
    candidate_vectors = np.array(
        chunks_vector_store.index.reconstruct_batch(candidate_indices[0])
    )
    candidate_texts = [idx_to_text(idx) for idx in candidate_indices[0]]
    
    # 计算距离矩阵
    # 使用 1 - 余弦相似度作为距离度量
    document_distances = 1 - np.dot(candidate_vectors, candidate_vectors.T)
    query_distances = 1 - np.dot(query_embedding, candidate_vectors.T)
    
    # 应用 dartboard 选择算法
    selected_texts, selection_scores = greedy_dartsearch(
        query_distances,
        document_distances,
        candidate_texts,
        num_results
    )
    
    return selected_texts, selection_scores

### dartboard 检索 - 在相同查询、k 和数据集上的结果
- 如您所见，现在前 3 个结果不再是简单的重复。

In [22]:
texts,scores=get_context_with_dartboard(test_query,k=3)
show_context(texts)


Context 1:
driven by human activities, particularly the emission of greenhou se gases.  
Chapter 2: Causes of Climate Change  
Greenhouse Gases  
The primary cause of recent climate change is the increase in greenhouse gases in the 
atmosphere. Greenhouse gases, such as carbon dioxide (CO2), methane (CH4), and nitrous 
oxide (N2O), trap heat from the sun, creating a "greenhouse effect." This effect is  essential 
for life on Earth, as it keeps the planet warm enough to support life. However, human 
activities have intensified this natural process, leading to a warmer climate.  
Fossil Fuels  
Burning fossil fuels for energy releases large amounts of CO2. This includes coal, oil, and 
natural gas used for electricity, heating, and transportation. The industrial revolution marked 
the beginning of a significant increase in fossil fuel consumption, which continues to rise 
today.  
Coal


Context 2:
Most of these climate changes are attributed to very small variations in Earth's orbit tha